In [1]:
%env JAVA_HOME=/home/tmpdeatheye/.sdkman/candidates/java/current

env: JAVA_HOME=/home/tmpdeatheye/.sdkman/candidates/java/current


In [2]:
from delta import *
from delta.tables import *

In [3]:
conf = {
    "spark.hadoop.fs.s3a.impl": "org.apache.hadoop.fs.s3a.S3AFileSystem",
    "spark.hadoop.fs.s3a.endpoint": "http://10.1.1.2:9000",
    "spark.hadoop.fs.s3a.access.key": "username",
    "spark.hadoop.fs.s3a.secret.key": "password",
    "spark.hadoop.fs.s3a.path.style.access": "true",
    "spark.sql.catalogImplementation": "hive",
    "spark.hive.metastore.uris": "thrift://10.1.1.7:9083",
    "spark.hive.metastore.schema.verification": "false",
    "spark.sql.extensions": "io.delta.sql.DeltaSparkSessionExtension",
    "spark.sql.catalog.spark_catalog": "org.apache.spark.sql.delta.catalog.DeltaCatalog",
    "spark.sql.debug.maxToStringFields": "1024"
}

packages = [
    "org.apache.spark:spark-avro_2.12:3.5.1",
    "org.apache.hadoop:hadoop-aws:3.3.4",
    "io.delta:delta-spark_2.12:3.1.0"
]

spark = configure_spark_with_delta_pip(
    spark_session_builder=SparkSession.builder.config(map=conf),
    extra_packages=packages
).getOrCreate()

24/03/27 07:33:29 WARN Utils: Your hostname, ubuntu resolves to a loopback address: 127.0.1.1; using 192.168.2.8 instead (on interface wlp0s20f3)
24/03/27 07:33:29 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/opt/miniconda3/envs/venv/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/tmpdeatheye/.ivy2/cache
The jars for the packages stored in: /home/tmpdeatheye/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
org.apache.spark#spark-avro_2.12 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-84d358c3-fd5b-4ffe-b170-4c0ea981a522;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.1.0 in central
	found io.delta#delta-storage;3.1.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in local-m2-cache
	found org.apache.spark#spark-avro_2.12;3.5.1 in central
	found org.tukaani#xz;1.9 in spark-list
	found org.apache.hadoop#hadoop-aws;3.3.4 in local-m2-cache
	found com.amazonaws#aws-java-sdk-bundle;1.12.262 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in local-m2-cache
:: resolution report :: resolve 182ms :: artifacts dl 5ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-b

In [4]:
spark.sql("""
    CREATE SCHEMA billing LOCATION 's3a://lakehouse/bronze/billing/'
""")

DataFrame[]

In [5]:
spark.sql("""
    CREATE TABLE billing.invoice (
        order_id BIGINT,
        created_at TIMESTAMP,
        description STRING,
        date DATE
    )
    USING DELTA
    PARTITIONED BY (date)
    LOCATION 's3a://lakehouse/bronze/billing/main/raw/invoice/'
""")

24/03/27 00:18:57 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
24/03/27 00:19:01 WARN HiveExternalCatalog: Couldn't find corresponding Hive SerDe for data source provider delta. Persisting data source table `spark_catalog`.`billing`.`invoice` into Hive metastore in Spark SQL specific format, which is NOT compatible with Hive.
24/03/27 00:19:01 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.


DataFrame[]

In [12]:
spark.sql("""
    SELECT * FROM billing.invoice
""").show(truncate=False)

+--------+-------------------+-----------+----------+
|order_id|created_at         |description|date      |
+--------+-------------------+-----------+----------+
|2       |2024-03-27 00:28:06|two-two    |2024-03-26|
+--------+-------------------+-----------+----------+
